In [ ]:
import os
import xarray as xr
import numpy as np
import glob
import geopandas as gpd

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors as mcol
from matplotlib.lines import Line2D

import pandas as pd
from mpl_toolkits.axes_grid1 import make_axes_locatable 

In [ ]:
# Open the background files
rhine = gpd.read_file(r'C:\Users\Fonse_ma\OneDrive - Stichting Deltares\Desktop\Maria\Thesis\GIS\Python\Rhine.geojson')
regions = gpd.read_file(r'C:\Users\Fonse_ma\OneDrive - Stichting Deltares\Desktop\Maria\Thesis\GIS\Python\Rhine_Regions.geojson')
mregions = gpd.read_file(r'C:\Users\Fonse_ma\OneDrive - Stichting Deltares\Desktop\Maria\Thesis\GIS\Python\Rhine_MainRegions.geojson')
countries = gpd.read_file(r'C:\Users\Fonse_ma\OneDrive - Stichting Deltares\Desktop\Maria\Thesis\GIS\Python\Rhine_Countries.geojson')
confluences = gpd.read_file(r'C:\Users\Fonse_ma\OneDrive - Stichting Deltares\Desktop\Maria\Thesis\GIS\Python\Rhine_Confluences.geojson')
rivers = gpd.read_file(r'C:\Users\Fonse_ma\OneDrive - Stichting Deltares\Desktop\Maria\Thesis\GIS\Python\Rhine_River.geojson')
trib = gpd.read_file(r'C:\Users\Fonse_ma\OneDrive - Stichting Deltares\Desktop\Maria\Thesis\GIS\Python\Rhine_Tributaries.geojson')
mriver= gpd.read_file(r'C:\Users\Fonse_ma\OneDrive - Stichting Deltares\Desktop\Maria\Thesis\GIS\Python\Rhine_MainRiver.geojson')
sbasin = gpd.read_file(r'C:\Users\Fonse_ma\OneDrive - Stichting Deltares\Desktop\Maria\Thesis\GIS\Python\Rhine_Subbasins.geojson')


In [ ]:
#MET
# Open ALL the dataframes
domain = 'MET'
desc = 'rain5d' #OPEN 'rain1d', 'snow', 'rain5d', (ONLY N=1000)'r50'
extra = 'r5d' #RUN 'r1d', 's5d', 'r5d', (ONLY N=1000)'r50'
for A in (1, 2, 3):
    for n in (100, 500, 1000, 5000, 10000, 25000, 50000): #5000, 10000, 25000, 
        op = r"P:\11206883-006-dar-cloud-computing\RhineFiles\MET\04_METmetrics\N%d" %n + r"_AM%d_" %A + desc + r".pkl"
        globals()[f'{domain}_N{n}_AM{A}_{extra}'] = pd.read_pickle(op)
        
    for n in (20, 50): #5000, 10000, 25000, 
        op = r"P:\11206883-006-dar-cloud-computing\RhineFiles\MET\04_METmetrics\N%d" %n + r"_AM%d_" %A + desc + r".pkl"
        globals()[f'{domain}_N{n}_AM{A}_{extra}'] = pd.read_pickle(op)
#         globals()[f'{domain}_N{n}_AM{A}_{extra}'].index = globals()[f'{domain}_N{n}_AM{A}_{extra}'].Confluence

In [ ]:
domain = 'MET' # 'HYD', 'MET'
extra = 'r5d' #RUN 'r1d', 's5d', 'r5d', (ONLY N=1000)'r50'
for A in (1, 2, 3):
    for n in (20, 50, 100, 500, 1000, 5000, 10000, 25000, 50000): #, 1000, 5000, 10000, 25000, 50000):
        df = globals()[f'{domain}_N{n}_AM{A}_{extra}']
        globals()[f'g{domain}_N{n}_AM{A}_{extra}'] = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.X, df.Y), crs="EPSG:32632")

In [ ]:
#For later HYD
#HYD
# Open ALL the dataframes
domain = 'HYD'
desc = 'disc1d' #OPEN 'disc1d', 'disc5d'
extra = 'd1d' #RUN 'd1d', 'd5d'
for A in (1, 2, 3):
#     n = 10000
    for n in (20, 50, 100, 500, 1000, 5000, 50000): #5000, 10000, 25000,
#         op = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HID\HIDmetrics\C_AM%d" %A + r"_n%d" %n  + desc + r".pkl"
        op = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HIDnew\04_HYDmetrics\N%d" %n + r"_AM%d_" %A + desc + r".pkl"
        globals()[f'{domain}_N{n}_AM{A}_{extra}'] = pd.read_pickle(op)
        globals()[f'{domain}_N{n}_AM{A}_{extra}'].index = globals()[f'{domain}_N{n}_AM{A}_{extra}'].Conf

In [ ]:
domain = 'HYD' # 'HYD', 'MET'
extra = 'd1d' #RUN 'r1d', 's5d', 'r5d', (ONLY N=1000)'r50'
for A in (1, 2, 3):
    for n in (20, 50, 100, 500, 1000, 5000, 50000):
#     n = 10000
        df = globals()[f'{domain}_N{n}_AM{A}_{extra}']
        globals()[f'g{domain}_N{n}_AM{A}_{extra}'] = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.X, df.Y), crs="EPSG:32632")

## Differences between the AMSets --> compared to SET1

In [ ]:
def plot3difAM(gdfAM1, gdfAM2, gdfAM3, title, n, var, cmap, vmin, vmax):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 10))
    fig.suptitle(f'{title}', fontsize=20) # | N={n} years

    ax1.set_title('Set 1 | MSmax-TSconc', fontsize=16)
#     regions.plot(ax=ax1, color='white', edgecolor='black', figsize=(20, 10), zorder=0)
#     sbasin.plot(ax=ax1, color='white', edgecolor='black', linewidth=0.5, alpha=0.6, zorder=1)
    mregions.plot(ax=ax1, color='white', edgecolor='black', figsize=(20, 10), zorder=0, label='Regions') #NEW
    trib.plot(ax=ax1, color='blue', linewidth=0.8, alpha=0.8, zorder=1, label='Tributary Rivers') #NEW
    mriver.plot(ax=ax1, color='blue', linewidth=1, alpha=1, zorder=1)
    rivers.plot(ax=ax1, color='blue', linewidth=0.4, alpha=0.4, zorder=1)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    gdfAM1.plot(gdfAM1[var], ax=ax1, marker='o', markersize=80, edgecolors='k', cmap=cmap, zorder=2, vmin=vmin, vmax=vmax, legend=True, legend_kwds={'label': f"Spearman's r_s",
                            'orientation': "vertical"}, cax=cax)
    #TEXT
    rmin = gdfAM1[var].min()
    rmax = gdfAM1[var].max()
    props = dict(boxstyle='square', facecolor='white')
    ax1.text(580000, 5960000, f"r_s min = {rmin:.2f} \nr_s max = {rmax:.2f}", fontsize=12, ha='center', verticalalignment='top', bbox=props)


    
    vmax= 0.3
    vmin=-vmax
    #AM2
    ax2.set_title('Set 2 | TSmax-MSconc', fontsize=16)
#     regions.plot(ax=ax2, color='white', edgecolor='black', figsize=(20, 10), zorder=0)
#     sbasin.plot(ax=ax2, color='white', edgecolor='black', linewidth=0.5, alpha=0.6, zorder=1)
    # countries.plot(ax=ax, color='white', edgecolor='black')
    mregions.plot(ax=ax2, color='white', edgecolor='black', figsize=(20, 10), zorder=0, label='Regions') #NEW
    trib.plot(ax=ax2, color='blue', linewidth=0.8, alpha=0.8, zorder=1, label='Tributary Rivers') #NEW
    mriver.plot(ax=ax2, color='blue', linewidth=1, alpha=1, zorder=1)
    rivers.plot(ax=ax2, color='blue', linewidth=0.4, alpha=0.4, zorder=1)
    divider = make_axes_locatable(ax2)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    gdfAM2.plot(gdfAM1[var]-gdfAM2[var], ax=ax2, marker='o', markersize=80, edgecolors='k', cmap='PiYG', zorder=2, vmin=vmin, vmax=vmax, legend=True, legend_kwds={'label': f"Set1 - Set2 | Spearman's r_s",
                            'orientation': "vertical"}, cax=cax) #RdGy_r
    #TEXT CHECK
    rmin = np.min(gdfAM1[var]-gdfAM2[var])
    rmax = np.max(gdfAM1[var]-gdfAM2[var])
    props = dict(boxstyle='square', facecolor='white')
#     ax.text(470000, 5960000, f"{var} min = {rmin:.4f} \n{var} max = {rmax:.4f}", fontsize=14, verticalalignment='top', bbox=props)
    ax2.text(580000, 5960000, f"r_s min = {rmin:.2f} \nr_s max = {rmax:.2f}", fontsize=12, ha='center', verticalalignment='top', bbox=props)


    #AM3
    ax3.set_title('Set 3 | Cmax (C=MS+TS)', fontsize=16)
#     regions.plot(ax=ax3, color='white', edgecolor='black', linewidth=0.5, alpha=0.6, figsize=(20, 10), zorder=0)
#     sbasin.plot(ax=ax3, color='white', edgecolor='black', linewidth=0.5, alpha=0.6, zorder=1)
    # countries.plot(ax=ax, color='white', edgecolor='black')
    # confluences.plot(ax=ax, color='red', zorder=3)
    mregions.plot(ax=ax3, color='white', edgecolor='black', figsize=(20, 10), zorder=0, label='Regions') #NEW
    trib.plot(ax=ax3, color='blue', linewidth=0.8, alpha=0.8, zorder=1, label='Tributary Rivers') #NEW
    mriver.plot(ax=ax3, color='blue', linewidth=1, alpha=1, zorder=1)
    rivers.plot(ax=ax3, color='blue', linewidth=0.4, alpha=0.4, zorder=1)
    divider = make_axes_locatable(ax3)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    gdfAM3.plot(gdfAM1[var]-gdfAM3[var], ax=ax3, marker='o', markersize=80, edgecolors='k', cmap='PiYG', zorder=2, legend=True, vmin=vmin, vmax=vmax, legend_kwds={'label': f"Set1 - Set2 | Spearman's r_s",
                            'orientation': "vertical"}, cax=cax) #RdGy_r
    #TEXT CHECK
    rmin = np.min(gdfAM1[var]-gdfAM3[var])
    rmax = np.max(gdfAM1[var]-gdfAM3[var])
    props = dict(boxstyle='square', facecolor='white')
#     ax.text(470000, 5960000, f"{var} min = {rmin:.4f} \n{var} max = {rmax:.4f}", fontsize=14, verticalalignment='top', bbox=props)
    ax3.text(580000, 5960000, f"r_s min = {rmin:.2f} \nr_s max = {rmax:.2f}", fontsize=12, ha='center', verticalalignment='top', bbox=props)


    
    plt.tight_layout()
    plt.show()
    return

In [ ]:
# extra = 'r5' #RUN 'r1', 's', 'r5', (ONLY N=1000)'r50'
domain = 'MET'
extra = 'r5d' 

cmap = 'hot_r'#'Reds' #'viridis_r'
var = 'rho_mean' #std
N = [50000] #100, 500, 1000, 5000, 10000, 25000, 
# vmin, vmax = minmax(var, domain, extra, N)
vmin = 0
vmax = 1




for n in N:
    title = f'Correlation between Annual Maxima peaks of the Mainstream and the Tributary stream \n Differences between the Annual Maxima Sets  \n Precipitation 5 days N={n}' #  f'{domain} | Precipitation 5d N={n} \n Differences of rho between AM Sets'
    plot3difAM(globals()[f'g{domain}_N{n}_AM1_{extra}'], globals()[f'g{domain}_N{n}_AM2_{extra}'], globals()[f'g{domain}_N{n}_AM3_{extra}'], title, n, var, cmap, vmin, vmax)
    
domain = 'HYD'
extra = 'd1d' 

# cmap = 'Reds' #'viridis_r'
var = 'rho_mean' #std
N = [50000] #100, 500, 1000, 5000, 10000, 25000, 
# vmin, vmax = minmax(var, domain, extra, N)
vmin = 0
vmax = 1



for n in N:
    title = f'Correlation between Annual Maxima peaks of the Mainstream and the Tributary stream \n Differences between the Annual Maxima Sets \n Discharge 1 day N={n}' # f'{domain} | Discharge 1d N={n} \n Differences of rho between AM Sets'
    plot3difAM(globals()[f'g{domain}_N{n}_AM1_{extra}'], globals()[f'g{domain}_N{n}_AM2_{extra}'], globals()[f'g{domain}_N{n}_AM3_{extra}'], title, n, var, cmap, vmin, vmax)
    